<a name='1'></a>
## Import Packages and load data

In [1]:
### -*- coding: utf-8 -*-
"""
Created on Mon May 24 13:26:13 2021

@author: kjsanche

Description: 
A function to load the 5 minute granules from MODIS channel 1 
(0.65 microns) and the contrail mask for ML with a CNN.

To do:
ASAP:
-cut up images
-separate testing data
-plot testing data vs mask



lower priority:
-optimize image shape by transforming back to original satellite swath projection
-save and set up to load data as tfrecord
-organize/markdown/comment code


Input:
Path   (string)

        
        
Output:
MODISCh1 (2D uint32)
MASK     (2D uint16)
"""


from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import struct
import os
import glob
import cv2
import gc
from UNET_Functions import unet_model, summary
from Sat_contrail_read import Extract_RawDef, extract_img, extract_mask, extract_imglist, get_model_memory_usage
from scipy.spatial import ConvexHull
from scipy.ndimage import rotate
from format_input import *
import tensorflow as tf

sys_details = tf.sysconfig.get_build_info()
print(sys_details)
cudnn_version = sys_details["cudnn_version"]
cuda_version = sys_details["cuda_version"]

print('cuda version: ', cuda_version)
print('cudNN version: ',cudnn_version)
print('TF version: ', tf.version.VERSION)


NEWMASK = True # use updated masks?
NCHANNELS=7 #number of channels + 1 (3, 7 or 8)
DVERSION = 'v26' #v1 = 7 channels, old masks; v2 = 3 channels, old mask; v3 = 7 channels, new mask; v4 = 3 channels, new mask
#augment_amount = 0
img_slice_size = 64
contrail_ratio = 3 #number of slices to keep without contrails:slices with contrails (2 = 2:1)
pixel_threshold = (img_slice_size**2)*0.005 #pixel threshold to determine if slice has a significant number of pixels labeled as contrail


#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.experimental.output_all_intermediates(True) 
path = os.getcwd()
proj_path = os.path.normpath(path + os.sep + os.pardir) #get parent directory
image_path = '/home/kjsanche/Desktop/ExternalSSD/SatContrailData/' #os.path.join(proj_path, 'data/')
save_TFrecord_path = '/home/kjsanche/Desktop/ExternalSSD/SatContrailData/TFrecords/' 


image0065, image0380, image0680, image0850, image1100, image1200, image1330, AUX_list, mask_list = extract_imglist(image_path, NEWMASK, NCHANNELS)

N = len(image1200)
print(N)
print(len(mask_list))

OrderedDict([('cpu_compiler', '/home/builder/ktietz/aggregate/tensorflow_recipes/ci_cpu/tensorflow-base_1614583966145/_build_env/bin/x86_64-conda_cos6-linux-gnu-gcc'), ('cuda_compute_capabilities', ['compute_35', 'compute_52', 'compute_60', 'compute_61', 'compute_70', 'compute_75']), ('cuda_version', '10.1'), ('cudnn_version', '7'), ('is_cuda_build', True), ('is_rocm_build', False)])
cuda version:  10.1
cudNN version:  7
TF version:  2.4.1
105
105


## extract image dimensions

In [2]:
dim = Extract_RawDef(AUX_list)

## make dataset

In [3]:
img0065_filenames = tf.constant(image0065)
img0380_filenames = tf.constant(image0380)
img0680_filenames = tf.constant(image0680)
img0850_filenames = tf.constant(image0850)
img1100_filenames = tf.constant(image1100)
img1200_filenames = tf.constant(image1200)
img1330_filenames = tf.constant(image1330)

masks_filenames = tf.constant(mask_list)
print(img1100_filenames.shape)
print(masks_filenames.shape)
dataset = tf.data.Dataset.from_tensor_slices((img0065_filenames, img0380_filenames, img0680_filenames, img0850_filenames, img1100_filenames, img1200_filenames, img1330_filenames, masks_filenames, dim))
#print(dataset)

(105,)
(105,)


In [4]:
def process_path(img0065_path, img0380_path, img0680_path, img0850_path, img1100_path, img1200_path, img1330_path, mask_path,dim):
# convert binary files to matrix of integers
    if NCHANNELS >=3:
        img5 = extract_img(str(img1100_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img6 = extract_img(str(img1200_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img = np.concatenate((img5, img6), axis=2)
    if NCHANNELS >=7:
        img2 = extract_img(str(img0380_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img3 = extract_img(str(img0680_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img4 = extract_img(str(img0850_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img7 = extract_img(str(img1330_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img = np.concatenate((img2, img3, img4, img7, img), axis=2)
    if NCHANNELS ==8:
        img1 = extract_img(str(img0065_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img = np.concatenate((img1, img), axis=2)



    #img = np.concatenate((img1, img2, img3, img4, img7, img5, img6), axis=2)
    
    
    if NEWMASK:
        with open(str(mask_path.numpy().decode('ascii')), 'rb') as f:
            mask = np.zeros((4096,4096,1), dtype = int)
            x = np.load(f)
            mask[0:x.shape[0], 0:x.shape[1], 0] += x
    else:
        mask = extract_mask(str(mask_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        
    return tf.convert_to_tensor(img,dtype=tf.float16), tf.convert_to_tensor(mask,dtype=tf.int8)

def _set_shapes(img, mask):
    img.set_shape([4096, 4096, NCHANNELS-1])
    mask.set_shape([4096, 4096, 1])

    return img, mask 

#print(dataset)
image_ds = dataset.map(lambda aa, bb, cc, dd, ee, ff, gg, hh, ii: tf.py_function(process_path, [aa, bb, cc, dd, ee, ff, gg, hh, ii], [tf.float16, tf.int8]))
image_ds = image_ds.map(_set_shapes)
print(image_ds)



<MapDataset shapes: ((4096, 4096, 6), (4096, 4096, 1)), types: (tf.float16, tf.int8)>


In [5]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        print(i)
        print(display_list[i].shape)
        if i == 0:
            plt.imshow(np.float32(display_list[i][:,:,-3]-display_list[i][:,:,-2]), cmap = 'gray')
        else:
            plt.imshow(np.float32(1*display_list[i][:,:,0]), cmap = 'gray') 
        plt.axis('off')
    plt.show()

In [6]:
for image, mask in image_ds.take(5):
    sample_image, sample_mask = image, mask
print(sample_image.shape)
print(sample_mask.shape)


(4096, 4096, 6)
(4096, 4096, 1)


In [7]:
def rotate_img(image, angle):
    rotated_image =  rotate(np.float32(image), angle=angle, reshape=True, order=0)
    #zoom into minimum bounding rectangle
    if np.max(rotated_image)>0:
        y_min, y_max, x_min, x_max = [np.min(np.nonzero(rotated_image)[0][:]), np.max(np.nonzero(rotated_image)[0][:]), np.min(np.nonzero(rotated_image)[1][:]), np.max(np.nonzero(rotated_image)[1][:])]
        return rotated_image, y_min, y_max, x_min, x_max
    return rotated_image, np.nan, np.nan, np.nan, np.nan

In [8]:
#used to find range of values for each channel
def Rng_Extremes(x, rng):
    mx = np.max(np.max(x))
    x[x==0]=99999
    mn = np.min(np.min(x))
    x[x==99999]=0
    if rng == []:
        return [mn, mx]
    if mn<rng[0]: 
        mn = mn
    else:
        mn = rng[0]
    if mx>rng[1]: 
        mx = mx
    else:
        mx = rng[1]
    return [mn, mx]
        
c1rng =[]
c2rng =[]
c3rng =[]
c4rng =[]
c5rng =[]
c6rng =[]
c7rng =[]

In [9]:
def blockshaped(arr, nrows, ncols):
    """
    Return an array of shape (n, nrows, ncols) where
    n * nrows * ncols = arr.size

    If arr is a 2D array, the returned array should look like n subblocks with
    each subblock preserving the "physical" layout of arr.
    """
    h, w = arr.shape
    assert h % nrows == 0, f"{h} rows is not evenly divisible by {nrows}"
    assert w % ncols == 0, f"{w} cols is not evenly divisible by {ncols}"
    return (arr.reshape(h//nrows, nrows, -1, ncols)
               .swapaxes(1,2)
               .reshape(-1, nrows, ncols))
def unblockshaped(arr, h, w):
    """
    Return an array of shape (h, w) where
    h * w = arr.size

    If arr is of shape (n, nrows, ncols), n sublocks of shape (nrows, ncols),
    then the returned array preserves the "physical" layout of the sublocks.
    """
    n, nrows, ncols = arr.shape
    return (arr.reshape(h//nrows, -1, nrows, ncols)
               .swapaxes(1,2)
               .reshape(h, w))

In [10]:



fileList=tf.io.gfile.glob([save_TFrecord_path + '*' + DVERSION +'.tfrecords'])
cnt = 0
maskTPcount=np.zeros(np.shape(mask_list), dtype = float)

for image, mask in image_ds:
    sample_image, sample_mask = image, mask
    filename = mask_list[cnt][-36:-14] + DVERSION
    if NEWMASK:
        filename=mask_list[cnt][-36-len('update'):-14-len('update')] + DVERSION #different names for cases with updatedmask
    print(filename)
    maskTPcount[cnt]= 0
    cnt+=1
    
    #skip file if file already exists or nans/inf values are in images
    if save_TFrecord_path+'1_1' + filename + '.tfrecords' in fileList:
        continue

    if np.any(np.isnan(sample_image)) or np.any(np.isinf(sample_image)):
        #seems some values in sample_image[:,:,0] are inf sometimes
        continue #sys.exit()

    if np.any(np.isnan(sample_mask)) or np.any(np.isinf(sample_mask)):
        #print('mask')
        continue #sys.exit()


    #find coordinates of all nonzero coordinates
    nonzero =np.asarray(np.nonzero(np.float32(sample_image[:,:,-2]))).T
    if len(nonzero)==0:
        #at least 1 case is all zeros and breaks code
        continue

    #extract convexhull coordinates, first point and last point must be the same. 
    hull = ConvexHull(nonzero)
    hull = np.vstack((nonzero[hull.vertices,0],nonzero[hull.vertices,1])).T
    hull = np.vstack((hull,hull[0,:]))

    #calculate angle of rotation for minimum bounding rectangle
    angle, barea, bwidth, bheight, center_point, corner_points = minBoundingRect(hull)
    
    rotated_image, y_min, y_max, x_min, x_max = rotate_img(sample_image, angle=90-angle*180/np.pi)
    rotated_labels, _, _, _, _ = rotate_img(sample_mask, angle=90-angle*180/np.pi)
    
    w = x_max-x_min
    h = y_max-y_min
    
    #determine min/max dimensions
    maxCaseDim = np.max([w,h])
    minCaseDim = np.min([w,h])

    #rotate 90 degrees to make width the bigger side if needed
    if h > w:
        rotated_image, y_min, y_max, x_min, x_max = rotate_img(rotated_image, angle=90)
        rotated_image_zoom = rotated_image[y_min:y_max,x_min:x_max]
        rotated_labels, _, _, _, _ = rotate_img(rotated_labels, angle=90)
        rotated_labels_zoom = rotated_labels[y_min:y_max,x_min:x_max]
    else:
        rotated_image_zoom = rotated_image[y_min:y_max,x_min:x_max]
        rotated_labels_zoom = rotated_labels[y_min:y_max,x_min:x_max]
 
    #make images and mask have same h x w (2048,4096)
    mask = np.zeros((2048,4096,1), dtype = float)
    mask[0:rotated_labels_zoom.shape[0], 0:rotated_labels_zoom.shape[1]] += rotated_labels_zoom


    img_rotated = np.zeros((2048,4096,NCHANNELS), dtype = float)
    img_rotated[0:rotated_image_zoom.shape[0], 0:rotated_image_zoom.shape[1], :NCHANNELS-1] += rotated_image_zoom[:,:,1-NCHANNELS:]
    # set last channel (valid channel) to 1 for the pixels containing part of the satellite image (keep 0 for the blank pixels)
    img_rotated[img_rotated[:,:,-2] > 0 ,-1] = 1 

    #check min and max values of channels
    '''
    c1rng = Rng_Extremes(img_test[:,:,0],c1rng)
    c2rng = Rng_Extremes(img_test[:,:,1],c2rng)
    c3rng = Rng_Extremes(img_test[:,:,2],c3rng)
    c4rng = Rng_Extremes(img_test[:,:,3],c4rng)
    c5rng = Rng_Extremes(img_test[:,:,4],c5rng)
    c6rng = Rng_Extremes(img_test[:,:,5],c6rng)
    c7rng = Rng_Extremes(img_test[:,:,6],c7rng)
    '''
   

    img_rotated = np.float64(img_rotated)
    #resize images to decreased resolution
    #img_test = cv2.resize(img_test, dsize=(1024, 512), interpolation=cv2.INTER_NEAREST)
    #mask = cv2.resize(mask, dsize=(1024, 512), interpolation=cv2.INTER_NEAREST)

    #add missing dimensions
    img_rotated = np.expand_dims(img_rotated, axis=0)
    mask = np.expand_dims(mask, axis=0)

    #set zeros to match min value in pixel range (min value is approximately 0 in first three channels)
    if NCHANNELS-5>0:
        tmp = img_rotated[:,:,:,NCHANNELS-5:]
        tmp[tmp==0]=17000
        img_rotated[:,:,:,NCHANNELS-5:]=tmp
        BT_rng = np.float64([17000, 34000]) #this range is good for 4 upper channels. assuming range of 170-340 K (BT is multiplied by 100)
        img_rotated[:,:,:,NCHANNELS-5:-1] = (img_rotated[:,:,:,NCHANNELS-5:-1]-np.mean(BT_rng)) / ((BT_rng[1]-BT_rng[0])/2) 
        BT_rng = np.float64([0, 66535]) #lower 3 channels have a much larger range.
        img_rotated[:,:,:,:NCHANNELS-5] = (img_rotated[:,:,:,:NCHANNELS-5]-np.mean(BT_rng)) / ((BT_rng[1]-BT_rng[0])/2)
    else:
        img_rotated[img_rotated==0]=17000
        BT_rng = np.float64([17000, 34000]) #this range is good for 4 upper channels. assuming range of 170-340 K (BT is multiplied by 100)
        img_rotated[:,:,:,:-1] = (img_rotated[:,:,:,:-1]-np.mean(BT_rng)) / ((BT_rng[1]-BT_rng[0])/2) 

    
    #normalize img based on channel ranges and set mask to binary values identifying contrails
    # for resized image
    #mask[mask>0.5]=1
    #mask[mask<=0.5]=0
    # for full resolution image
    mask[mask>1]=1
    
    #change type to decrease size
    image = np.float16(img_rotated)
    mask = np.int8(mask)
    #cut up image to smaller pieces
    
    

    
    #augment and write images to file
    maskTPcount[cnt-1]= sum(sum(mask))
    
    
#    if sum(sum(mask))>0:
#        write_images_to_tfr_long(img_test, mask, filename=filename, max_files=1, out_dir=save_TFrecord_path)
#        augment_case(mask, img_test, 5)

    
    #create matrix with slices of granule of size "img_slice_size"
    mask_slices = blockshaped(squeeze(mask), img_slice_size, img_slice_size)

    n_slices = mask_slices.shape[0]
    for i in range(image.shape[3]):
        img_1slice = blockshaped(squeeze(image[:,:,:,i]), img_slice_size, img_slice_size)

        if i == 0:
            img_slices = img_1slice
        elif i == 1:
            img_slices = np.stack((img_slices, img_1slice), axis=3)
        else:
            img_slices = np.concatenate((img_slices, img_1slice[:,:,:,np.newaxis]),axis=-1)
    ContrailPixelCnt=np.sum(np.sum(mask_slices,axis=1),axis=1)
    
    

    #balance data set by removing most cases where pixel_threshold is not met 
    indexContrail = np.argwhere(ContrailPixelCnt > pixel_threshold)
    if len(indexContrail)*contrail_ratio < len(ContrailPixelCnt): 
        remove_num = n_slices-len(indexContrail)*(contrail_ratio+1)
    else:
        remove_num = 0
    number_of_imgs = n_slices-len(indexContrail)
    indexnocontrail = np.delete(range(n_slices), indexContrail, 0) #array of slice index with no contrail
    random_indices = np.random.choice(indexnocontrail, size=remove_num, replace=False) 
    balanced_mask_slices = np.delete(mask_slices, arange(n_slices)[random_indices], 0)
    balanced_img_slices = np.delete(img_slices, arange(n_slices)[random_indices], 0)
    if balanced_mask_slices.shape[0] == 0:
        continue
    print(f"number of mask slices = {balanced_mask_slices.shape[0]}")
    print(f"number of img slices = {balanced_img_slices.shape[0]}")
    if sum(sum(balanced_mask_slices))>0:
        print(sum(sum(balanced_mask_slices)))
        #plt.hist(ContrailPixelCnt.flatten())
    


  
    # augment
    
    expanded_masks = np.expand_dims(balanced_mask_slices, axis=3)
    case = np.concatenate((balanced_img_slices, expanded_masks), axis=3)
    
        
    for i in range(7):
        aug_mask_img = case
        if i == 3:
            aug_mask_img = tf.image.flip_left_right(aug_mask_img)
        else:
            aug_mask_img = tf.image.rot90(aug_mask_img)

        aug_masks = np.int8(np.rint(aug_mask_img[:,:,:,-1]))
        aug_imgs = np.float16(aug_mask_img[:,:,:,:-1])
        balanced_img_slices = np.concatenate((balanced_img_slices, aug_imgs),axis=0)
        balanced_mask_slices = np.concatenate((balanced_mask_slices, aug_masks),axis=0)
        print(f"number of mask slices = {balanced_mask_slices.shape[0]}")
        print(f"number of img slices = {balanced_img_slices.shape[0]}")
    

#    for i in range(augment_amount):
#        aug_mask_img = data_augmentation(case)
#        aug_masks = np.int8(np.rint(aug_mask_img[:,:,:,-1]))
#        aug_imgs = np.float16(aug_mask_img[:,:,:,:-1])
#        balanced_img_slices = np.concatenate((balanced_img_slices, aug_imgs),axis=0)
#        balanced_mask_slices = np.concatenate((balanced_mask_slices, aug_masks),axis=0)
#        print(f"number of mask slices = {balanced_mask_slices.shape[0]}")
#        print(f"number of img slices = {balanced_img_slices.shape[0]}")
        
    write_images_to_tfr_long(balanced_img_slices, balanced_mask_slices, filename=filename, max_files=32, out_dir=save_TFrecord_path)
    
    
    del balanced_img_slices, balanced_mask_slices, img_slices, mask_slices, img_rotated
    gc.collect()


MOD021KM-A2018091.1700v25
MOD021KM-A2018091.0515v25
MOD021KM-A2018091.1520v25
number of mask slices = 342
number of img slices = 342
4648
number of mask slices = 684
number of img slices = 684
number of mask slices = 1026
number of img slices = 1026
number of mask slices = 1368
number of img slices = 1368
number of mask slices = 1710
number of img slices = 1710
number of mask slices = 2052
number of img slices = 2052
number of mask slices = 2394
number of img slices = 2394
number of mask slices = 2736
number of img slices = 2736

Using 86 shard(s) for 2736 files, with up to 32 samples per shard


100%|██████████| 86/86 [00:00<00:00, 98.14it/s] 



Wrote 2736 elements to TFRecord
MOD021KM-A2018091.1000v25
number of mask slices = 108
number of img slices = 108
1953
number of mask slices = 216
number of img slices = 216
number of mask slices = 324
number of img slices = 324
number of mask slices = 432
number of img slices = 432
number of mask slices = 540
number of img slices = 540
number of mask slices = 648
number of img slices = 648
number of mask slices = 756
number of img slices = 756
number of mask slices = 864
number of img slices = 864

Using 27 shard(s) for 864 files, with up to 32 samples per shard


100%|██████████| 27/27 [00:00<00:00, 93.80it/s]



Wrote 864 elements to TFRecord
MOD021KM-A2018091.0150v25
number of mask slices = 48
number of img slices = 48
300
number of mask slices = 96
number of img slices = 96
number of mask slices = 144
number of img slices = 144
number of mask slices = 192
number of img slices = 192
number of mask slices = 240
number of img slices = 240
number of mask slices = 288
number of img slices = 288
number of mask slices = 336
number of img slices = 336
number of mask slices = 384
number of img slices = 384

Using 12 shard(s) for 384 files, with up to 32 samples per shard


...

100%|██████████| 3/3 [00:00<00:00, 94.04it/s]


Wrote 96 elements to TFRecord


MOD021KM-A2018091.0210v25
MOD021KM-A2018091.0820v25
number of mask slices = 36
number of img slices = 36
240
number of mask slices = 72
number of img slices = 72
number of mask slices = 108
number of img slices = 108
number of mask slices = 144
number of img slices = 144
number of mask slices = 180
number of img slices = 180
number of mask slices = 216
number of img slices = 216
number of mask slices = 252
number of img slices = 252
number of mask slices = 288
number of img slices = 288

Using 9 shard(s) for 288 files, with up to 32 samples per shard


100%|██████████| 9/9 [00:00<00:00, 94.15it/s]


Wrote 288 elements to TFRecord


In [11]:
mask = np.expand_dims(mask, axis=3)
train_iter_5im = np.concatenate((img_test, mask), axis=3)


plt.figure(figsize=(10, 10))

for i in range(12):
    plt.subplot(4, 3, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    aug_img = data_augmentation(train_iter_5im)
    mask = aug_img[:,:,:,-1]
    img_test = aug_img[:,:,:,:-1]
    mask = np.int8(mask)
    mask[mask>0.5]=1
    mask[mask<=0.5]=0
   
    plt.imshow(aug_img[0,:,:,6], cmap='gray')
    
    #plt.xlabel(class_types [np.argmax(train_iter_5label)], fontsize=13)
plt.tight_layout()    
plt.show()
plt.imshow(aug_img[0,:,:,4], cmap='gray')

NameError: name 'img_test' is not defined

In [ ]:
plt.hist(maskTPcount,100)

In [ ]:
print(np.shape(x))
y= np.concatenate((x, x), axis=2)
z= np.concatenate((x, y), axis=2)
print(np.shape(y))
print(np.shape(z))

In [ ]:
np.shape(mask_list)

In [ ]:
sum(maskTPcount>100)

In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        print(i)
        print(display_list[i].shape)
        if i == 0:
            plt.imshow(np.float32(display_list[i][0,:,:,-3]-display_list[i][0,:,:,-2]))
        else:
            plt.imshow(np.float32(1*display_list[i][0,:,:]))
        plt.axis('off')
    plt.show()

print(img_test.shape)
print(mask.shape)
#mask[img_test[:,:,:,0] == 0] = 2
#test = img_test[:,:,:,0] < .1
print(img_test)
display([img_test,mask])

